<a href="https://colab.research.google.com/github/khorzhengyu/project518/blob/main/ML_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
#modules for data analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.ticker as ticker

#modules for model building
#algorithms for sampling
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

#modules for machine learning algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import collections
from sklearn import datasets, metrics, preprocessing


# modules for evaluation
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report,confusion_matrix
from xgboost import XGBClassifier
from xgboost import Booster
from xgboost import DMatrix

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# read the csv file
df_train= pd.read_csv('gdrive/My Drive/fraudTrain.csv')
df_test = pd.read_csv('gdrive/My Drive/fraudTest.csv')

In [5]:
#concatenating the two datasets together
df = pd.concat([df_train,df_test], ignore_index =True)

In [6]:
df.columns = df.columns.str.replace('_', '')

In [ ]:
# taking 90000 data for computation
df = df.sample(frac=1, random_state=2)
df = df.head(n=90000)
df.isfraud.value_counts()

In [8]:
# Adding a date variable in the format: YYYY-MM-DD
df['transdatetranstime'] = pd.to_datetime(df['transdatetranstime'])
df['transdate'] = df['transdatetranstime'].dt.date
df['transdate'] = pd.to_datetime(df['transdate'])

In [ ]:
df.head()

In [10]:
#sort by transaction date
df.sort_values(by = 'transdate', inplace = True)

In [11]:
# Adding 'recnum' as index
df['recnum'] = range(1, len(df) + 1)

In [ ]:
#remove the first unused column
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head()

In [ ]:
print('Rows     :', df.shape[0])
print('Columns  :', df.shape[1])
print('\nFeatures:\n', df.columns.tolist())
print('\nMissing values :', df.isnull().sum().values.sum())
print('\nUnique value:\n', df.nunique())

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# df.loc[df['cc_num'] == 630451534402]

In [ ]:
df.info()

In [ ]:
# Creating candidates variables ####

from copy import deepcopy

# Make a copy of DataFrame to avoid modifying the original data
df_var = deepcopy(df)

# Convert values to string type for selected columns
cols_convert = ['ccnum', 'merchlat','merchlong']
for item in cols_convert:
    df_var[item] = df_var[item].astype(str)

# Print data types of the columns in the modified DataFrame
print(df_var.dtypes)

In [16]:
# Make variable combos
df_var['card-merchant'] = df_var['ccnum'] + df_var['merchant']
#df_var['card-merchant-location'] = df_var['cc_num'] + df_var['merch_lat'] + df_var['merch_long']

In [ ]:
df_var.info()

In [18]:
df_var['ccnum'] = df_var['ccnum'].astype(int)
df_var['merchlat'] = df_var['merchlat'].astype(float)
df_var['merchlong'] = df_var['merchlong'].astype(float)

In [19]:
## Day-since variable
# Creating the function for the variable
def ds(dataframe, g1, g2, name):
#     'Helps with calculating the day since variables'
    day_since = dataframe.groupby(g1)[g1].first()
    day_since = day_since.rename_axis(['None' for i in range(len(g1))]).groupby(g2).diff()
    day_since.columns = [name]
    day_since = day_since.rename_axis(g1)
    day_since[name] = day_since[name].dt.days.fillna(0)
    day_since = day_since.reset_index()
    return day_since

In [ ]:
# Creating Day-Since Variable
start_daySince = pd.datetime.now()
day_card = ds(df_var, ['ccnum', 'transdate'], 'ccnum', 'card_daysSince')
print("done!", pd.datetime.now()-start_daySince)
day_card

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
day_card.head(10)

In [21]:
pd.reset_option('max_columns')
pd.reset_option('max_rows')

In [ ]:
# day_card

In [ ]:
from datetime import datetime
time_ds_all=datetime.now()
# Calculate the Days Since variables for the required columns
ds_cols = ['ccnum','merchant','card-merchant']

ds_dict={}
for col in ds_cols:
    curr_time=datetime.now()
    curr_name = 'daysSince_' + col

    # Calculate the days-since variable (ds) and assign it to a global variable (curr_name)
    vars()[curr_name] = ds(df_var, [col, 'transdate'], col, col+'_daysSince')
    ds_dict[curr_name] = vars()[curr_name] # Save results to a dictionary

    print("Done with:", col, "; Time:", datetime.now()-curr_time)

print("DONE!", datetime.now()-time_ds_all)

In [23]:
ds_dict.keys()
print(type(ds_dict.keys()))

<class 'dict_keys'>


In [24]:
# Create a copy of the main DataFrame to avoid modifying the original data
df_ds = df_var.copy()

# Merge the days-since variables with the main dataset
for item in ds_dict.keys():
    col_variable = item.split('_')[1]
    df_ds = pd.merge(df_ds, vars()[item], how='left', left_on=[col_variable,'transdate'], right_on=[col_variable,'transdate'])

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
df_ds.loc[df_ds['merchant'] == 'fraud_Kuvalis Ltd']

In [25]:
pd.reset_option('max_columns')
pd.reset_option('max_rows')

In [ ]:
df_ds.to_csv('df_daysSince.csv')

In [26]:
from datetime import datetime
# Create Columns for the Necessary Time Periods ##
#This makes new columns for the various time periods.
# Make a list of variable combinations to iterate through and create time-related variables
var_combos = ['ccnum','merchant','card-merchant']

# Create column names
time_list = [0,1,3,7,14,30]
time_joined =[]
for time in time_list:
    time_joined.append('join_ts_'+str(time))

start_copy = datetime.now()
df_var1 = deepcopy(df_var)
df_var2 = deepcopy(df_var)
print('copy time', datetime.now()-start_copy)

# Creating columns for time
start_loop=datetime.now()
for time in time_list:
    temp_endTime = 'join_ts_' + str(time)
    df_var2[temp_endTime] = df_var2['transdate'] + dt.timedelta(days = time)
print('first loop', datetime.now()-start_loop)

copy time 0:00:00.028406
first loop 0:00:00.010466


In [ ]:
# Create the Frequency Candidate Variables ##
start_loop2=datetime.now()
df_final = deepcopy(df_var.set_index('recnum'))

for item in var_combos:
    df_var3 = df_var1[['recnum','transdate',item]]
    temp_list = time_joined + [item]
    df_var4 = df_var2[temp_list + ['recnum']].copy()
    df_var4.rename(columns={'recnum':'recnum2'},inplace=True) # this causes a warning to arise

    df_temp = pd.merge(df_var3, df_var4, left_on=[item], right_on=[item])

    for time in time_list:
        temp_endTime = 'join_ts_' + str(time)
        df2_temp = df_temp[(df_temp['transdate'] <= df_temp[temp_endTime]) & (df_temp['recnum2'] <= df_temp['recnum'])]
        temp_groupby = df2_temp[['recnum','transdate']].groupby('recnum')
        temp_name = item + '_' + 'freq' + str(time) + '_'
        df_final = pd.merge(df_final, getattr(temp_groupby,'count')().add_prefix(temp_name), left_index=True, right_index=True, how='left')

print('second loop', datetime.now()-start_loop2)
print(len(df_final.columns))
df_final.head()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
df_final.loc[df_final['ccnum'] == 6011504998544485]

In [ ]:
pd.reset_option('max_columns')
pd.reset_option('max_rows')

In [ ]:
# Save the frequency variables
df_final.to_csv('df_frequency_vars.csv')

In [29]:
start_loop3= datetime.now()
groupbyvar_denom = deepcopy(var_combos)
days_numer = ['0','1']
days_denom = ['7','14','30']

for b in groupbyvar_denom:
    for c in days_numer:
        for d in days_denom:
            temp = d
            df_final[b + '_' + c + '_dayfreq' + '_div_' + d + '_dayfreq' + '_velchange'] = \
                df_final[b + '_freq' + c + '_transdate'] / \
                df_final[b + '_freq' + d + '_transdate'] / float(temp)
print('third loop', datetime.now() - start_loop3)

third loop 0:00:00.068177


In [ ]:
# Save the frequency and velocity change variables
df_final.to_csv('df_freq_velchange_vars.csv')

In [31]:
# Reset the index so that the Recnum returns to being a normal column
df_final.reset_index(inplace=True)

In [32]:
# Merge df_final (frequency and velocity change) and df_ds (days-since)
df_all_vars = pd.merge(df_final, df_ds, on=list(df_var.columns))

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows',None)

# df_all_vars.loc[df_all_vars['cc_num'] == 4841313716651064]

In [33]:
from typing import List
# Optimize Function
def optimize_floats(df: pd.DataFrame) -> pd.DataFrame:
    floats = df.select_dtypes(include=['float64']).columns.tolist()
    df[floats] = df[floats].apply(pd.to_numeric, downcast='float')
    return df


def optimize_ints(df: pd.DataFrame) -> pd.DataFrame:
    ints = df.select_dtypes(include=['int64']).columns.tolist()
    df[ints] = df[ints].apply(pd.to_numeric, downcast='integer')
    return df


def optimize_objects(df: pd.DataFrame, datetime_features: List[str]) -> pd.DataFrame:
    for col in df.select_dtypes(include=['object']):
        if col not in datetime_features:
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                df[col] = df[col].astype('category')
        else:
            df[col] = pd.to_datetime(df[col])
    return df



def optimize(df: pd.DataFrame, datetime_features: List[str] = []):
    return optimize_floats(optimize_ints(optimize_objects(df, datetime_features)))

In [34]:
# Make variable combos
main_dataset_filled = deepcopy(df)
main_dataset_filled['ccnum_|_merchant'] = main_dataset_filled['ccnum'].astype('str') + main_dataset_filled['merchant'].astype('str')
 # main_dataset_filled['ccnum_|_merchantlocation'] = main_dataset_filled['cc_num'].astype('str') + main_dataset_filled['merch_lat'].astype('str') + main_dataset_filled['merch_long'].astype('str')

In [35]:
var_combos = ['ccnum','merchant','ccnum_|_merchant']

In [36]:
import psutil
print('Memory percentage used',psutil.virtual_memory().percent)

Memory percentage used 23.8


In [37]:
time_joined =['join_ts1']
for num in time_list:
    time_joined.append('join_ts2_'+str(num))

In [38]:
# Making time variables
start=datetime.now()

time_list = [0,1,3,7,14,30]
main_dataset_filled['join_ts1']=main_dataset_filled['transdate']
# dt_i
for dt_i in time_list:
    time = 'join_ts2_'+str(dt_i)
    main_dataset_filled[time]=main_dataset_filled['transdate'] + dt.timedelta(dt_i)
print('Completed ',datetime.now()-start)

Completed  0:00:00.014351


In [39]:
df_var1 = main_dataset_filled.copy()
df_var2 = main_dataset_filled.copy()

In [ ]:
# 'Cardnum_|_zip_totalamount14_Amount2' = total amount for the past 14 days
# for the same cardnumber and zip

In [ ]:
# Total Amount Variables
start_loop2= datetime.now()
df_final = deepcopy(main_dataset_filled.set_index('recnum'))

for item in var_combos:
    df_var3 = df_var1[['recnum','transdate','amt',item]]
    temp_list = time_joined + [item]
    df_var4 = df_var2[temp_list + ['recnum','amt']].copy()
    df_var4.rename(columns={'recnum':'recnum2','amt':'amt2'},inplace=True) # this causes a warning to arise
#     df_var4['record2'] = df_var2['record'] # this causes a warning to arise

    df_temp = pd.merge(df_var3, df_var4, left_on=[item], right_on=[item])

    for time in time_list:
        temp_endTime = 'join_ts2_' + str(time)
#         df2_temp = df_temp[(df_temp['date'] >= df_temp['join_ts1']) & (df_temp['date'] <= df_temp[temp_endTime])] # Original from TA
        df2_temp = df_temp[(df_temp['transdate'] <= df_temp[temp_endTime]) & (df_temp['recnum2'] <= df_temp['recnum'])]

        temp_groupby = df2_temp[['recnum','amt2']].groupby('recnum')

        temp_name = item + '_' + 'totalamount' + str(time) + '_'
        df_final = pd.merge(df_final, getattr(temp_groupby,'sum')().add_prefix(temp_name), left_index=True, right_index=True, how='left')
#         break
#     break
print('second loop', datetime.now()-start_loop2)
print(len(df_final.columns))
df_final.head()

In [ ]:
# Median amount
start_loop2= datetime.now()
# df_final = deepcopy(main_dataset_filled.set_index('Recnum'))

for item in var_combos:
    df_var3 = df_var1[['recnum','transdate','amt',item]]
    temp_list = time_joined + [item]
    df_var4 = df_var2[temp_list + ['recnum','amt']].copy()
    df_var4.rename(columns={'recnum':'recnum2','amt':'amt2'},inplace=True)
#     df_var4['record2'] = df_var2['record'] # this causes a warning to arise

    df_temp = pd.merge(df_var3, df_var4, left_on=[item], right_on=[item])

    for time in time_list:
        temp_endTime = 'join_ts2_' + str(time)
#         df2_temp = df_temp[(df_temp['date'] >= df_temp['join_ts1']) & (df_temp['date'] <= df_temp[temp_endTime])] # Original from TA
        df2_temp = df_temp[(df_temp['transdate'] <= df_temp[temp_endTime]) & (df_temp['recnum2'] <= df_temp['recnum'])]

        temp_groupby = df2_temp[['recnum','amt2']].groupby('recnum')

        temp_name = item + '_' + 'median' + str(time) + '_'
        df_final = pd.merge(df_final, getattr(temp_groupby,'median')().add_prefix(temp_name), left_index=True, right_index=True, how='left')
#         break
#     break
print('second loop', datetime.now()-start_loop2)
print(len(df_final.columns))
df_final.head()

In [ ]:
# Mean Variables

start_loop2= datetime.now()
# df_final = deepcopy(main_dataset_filled.set_index('Recnum'))

for item in var_combos:
    df_var3 = df_var1[['recnum','transdate','amt',item]]
    temp_list = time_joined + [item]
    df_var4 = df_var2[temp_list + ['recnum','amt']].copy()
    df_var4.rename(columns={'recnum':'recnum2','amt':'amt2'},inplace=True)
#     df_var4['record2'] = df_var2['record'] # this causes a warning to arise

    df_temp = pd.merge(df_var3, df_var4, left_on=[item], right_on=[item])

    for time in time_list:
        temp_endTime = 'join_ts2_' + str(time)
#         df2_temp = df_temp[(df_temp['date'] >= df_temp['join_ts1']) & (df_temp['date'] <= df_temp[temp_endTime])] # Original from TA
        df2_temp = df_temp[(df_temp['transdate'] <= df_temp[temp_endTime]) & (df_temp['recnum2'] <= df_temp['recnum'])]

        temp_groupby = df2_temp[['recnum','amt2']].groupby('recnum')

        temp_name = item + '_' + 'mean' + str(time) + '_'
        df_final = pd.merge(df_final, getattr(temp_groupby,'mean')().add_prefix(temp_name), left_index=True, right_index=True, how='left')
#         break
#     break
print('second loop', datetime.now()-start_loop2)
print(len(df_final.columns))
df_final.head()

In [ ]:
 # Max Variables
start_loop2= datetime.now()
# df_final = deepcopy(main_dataset_filled.set_index('Recnum'))

for item in var_combos:
    df_var3 = df_var1[['recnum','transdate','amt',item]]
    temp_list = time_joined + [item]
    df_var4 = df_var2[temp_list + ['recnum','amt']].copy()
    df_var4.rename(columns={'recnum':'recnum2','amt':'amt2'},inplace=True)
#     df_var4['record2'] = df_var2['record'] # this causes a warning to arise

    df_temp = pd.merge(df_var3, df_var4, left_on=[item], right_on=[item])

    for time in time_list:
        temp_endTime = 'join_ts2_' + str(time)
#         df2_temp = df_temp[(df_temp['date'] >= df_temp['join_ts1']) & (df_temp['date'] <= df_temp[temp_endTime])] # Original from TA
        df2_temp = df_temp[(df_temp['transdate'] <= df_temp[temp_endTime]) & (df_temp['recnum2'] <= df_temp['recnum'])]

        temp_groupby = df2_temp[['recnum','amt2']].groupby('recnum')

        temp_name = item + '_' + 'max' + str(time) + '_'
        df_final = pd.merge(df_final, getattr(temp_groupby,'max')().add_prefix(temp_name), left_index=True, right_index=True, how='left')
#         break
#     break
print('second loop', datetime.now()-start_loop2)
print(len(df_final.columns))
df_final.head()

In [44]:
df_final = optimize(df_final,['transdate'])

In [45]:
all_columns = df_final.columns.tolist()
all_columns

['transdatetranstime',
 'ccnum',
 'merchant',
 'category',
 'amt',
 'first',
 'last',
 'gender',
 'street',
 'city',
 'state',
 'zip',
 'lat',
 'long',
 'citypop',
 'job',
 'dob',
 'transnum',
 'unixtime',
 'merchlat',
 'merchlong',
 'isfraud',
 'transdate',
 'ccnum_|_merchant',
 'join_ts1',
 'join_ts2_0',
 'join_ts2_1',
 'join_ts2_3',
 'join_ts2_7',
 'join_ts2_14',
 'join_ts2_30',
 'ccnum_totalamount0_amt2',
 'ccnum_totalamount1_amt2',
 'ccnum_totalamount3_amt2',
 'ccnum_totalamount7_amt2',
 'ccnum_totalamount14_amt2',
 'ccnum_totalamount30_amt2',
 'merchant_totalamount0_amt2',
 'merchant_totalamount1_amt2',
 'merchant_totalamount3_amt2',
 'merchant_totalamount7_amt2',
 'merchant_totalamount14_amt2',
 'merchant_totalamount30_amt2',
 'ccnum_|_merchant_totalamount0_amt2',
 'ccnum_|_merchant_totalamount1_amt2',
 'ccnum_|_merchant_totalamount3_amt2',
 'ccnum_|_merchant_totalamount7_amt2',
 'ccnum_|_merchant_totalamount14_amt2',
 'ccnum_|_merchant_totalamount30_amt2',
 'ccnum_median0_amt2'

In [ ]:
start_column = df_final.columns.get_loc('ccnum_totalamount0_amt2')
for col_names in df_final.columns[start_column:]:
#  the number of days in each column is the column index mod length of time array
    number_index = time_list[(df_final.columns.get_loc(col_names)%len(time_list))-1]
#     we split the name of the array to keep the first part and use it to create the new column names
    new_col_name = col_names.split(str(number_index))[0]+'_'+str(number_index)+'_actual'
#     print(new_col_name)
    df_final[new_col_name] = df_final["amt"]/df_final[col_names]

In [ ]:
# # df_final['Merchnum'][8]
# pd.set_option('display.max_rows', None)
# df_final[['ccnum']].loc[df_final['ccnum_|_merchant_totalamount_7_actual'] != 1]

# # np.array([df_final.columns == 'Merchnum_max_7_actual']).sum()
# # df_final['Merch']

In [ ]:
# Amount Velocity Change Variables
# iterates through  the variables created with the var_combos columns and finds the relative
# velocity variables. Divides the number of days of days_numer array with the days_denom one
start_loop3= datetime.now()
groupbyvar_denom = deepcopy(var_combos)
days_numer = ['0','1']
days_denom = ['7','14','30']
# ccnum_totalamount0_amt2_/_ccnum_totalamount3_amt2
for b in groupbyvar_denom:
    for c in days_numer:
        for d in days_denom:
            temp = d
            df_final[b + '_' + c + '_dayamount' + '_div_' + d + '_dayamount' + '_velchange'] = \
                df_final[b+'_totalamount'+c+'_amt2'] / \
                df_final[b+'_totalamount'+d+'_amt2'] / float(temp)
print('third loop', datetime.now() - start_loop3)

In [ ]:
pd.reset_option('max_columns')
pd.reset_option('max_rows')

In [48]:
df_all_vars = optimize(df_all_vars)

In [49]:
# df_all_vars contains frequency variables, velocity change variables, and days since variables
# merge1_df contains amount variables, velocity change amount variables, and benford's law variables
df_all_vars_final = pd.merge(df_all_vars, df_final, how='left',left_on=df.columns.to_list(),right_on=df.columns.to_list())

In [ ]:
df_all_vars_final = df_all_vars_final.drop(columns=['card-merchant', 'join_ts1', 'join_ts2_0',
       'join_ts2_1', 'join_ts2_3', 'join_ts2_7', 'join_ts2_14',
       'join_ts2_30'])
df_all_vars_final.head()

In [ ]:
#### Examples ########
#
#
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Selecting specific columns
selected_columns = ['ccnum', 'transdate', 'amt', 'ccnum_freq0_transdate', 'ccnum_freq1_transdate' ,'ccnum_freq3_transdate', 'ccnum_freq7_transdate']
filtered_df = df_all_vars_final[selected_columns]

# Filtering rows based on ccnum
ccnum_filter = 6011504998544485
filtered_df = filtered_df[filtered_df['ccnum'] == ccnum_filter]

# Define the range of indices you want to show
start_index = 5
end_index = 200

# Filter rows within the specified index range
filtered_df = filtered_df.iloc[start_index:end_index+1]

filtered_df


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Selecting specific columns
selected_columns = ['recnum', 'ccnum', 'merchant', 'transdate', 'amt', 'ccnum_totalamount0_amt2', 'ccnum_|_merchant_totalamount7_amt2' ,'ccnum_|_merchant_totalamount_7_actual']
filtered_df = df_all_vars_final[selected_columns]

# Filtering rows based on ccnum
ccnum_filter = 370877495212014
filtered_df = filtered_df[filtered_df['ccnum'] == ccnum_filter]

filtered_df


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Selecting specific columns
selected_columns = ['recnum', 'ccnum', 'transdate', 'amt', 'ccnum_totalamount0_amt2', 'ccnum_totalamount7_amt2' ,'ccnum_0_dayamount_div_7_dayamount_velchange']
filtered_df = df_all_vars_final[selected_columns]

# Filtering rows based on ccnum
ccnum_filter = 370877495212014
filtered_df = filtered_df[filtered_df['ccnum'] == ccnum_filter]

filtered_df


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Selecting specific columns
selected_columns = ['recnum', 'ccnum', 'transdate', 'amt', 'ccnum_freq0_transdate', 'ccnum_freq7_transdate' ,'ccnum_0_dayfreq_div_7_dayfreq_velchange']
filtered_df = df_all_vars_final[selected_columns]

# Filtering rows based on ccnum
ccnum_filter = 370877495212014
filtered_df = filtered_df[filtered_df['ccnum'] == ccnum_filter]

filtered_df


In [ ]:
pd.reset_option('max_columns')
pd.reset_option('max_rows')

In [ ]:
# print("Columns in df_final:", df_final.columns)
# print("Columns in df_all_var:", df_all_vars.columns)

In [ ]:
df = df_all_vars_final.copy()
df.head()

In [ ]:
results = pd.concat(
    [
        df['amt'].describe(percentiles=[0.5, 0.95, 0.999]).reset_index().rename(columns={'index': 'Row Type', 'amt': 'Overall Amt Distribution'}).round(3),
        df.loc[df['isfraud'] == 0, ['amt']].describe(percentiles=[0.5, 0.95, 0.999]).reset_index(drop=1).rename(columns={'amt': 'Non-Fraud Amt Distribution'}).round(3),
        df.loc[df['isfraud'] == 1, ['amt']].describe(percentiles=[0.5, 0.95, 0.999]).reset_index(drop=1).rename(columns={'amt': 'Fraud Amt Distribution'}).round(3)
    ], axis=1)
results

In [ ]:
# all_trans = df.copy()
# all_trans['class'] = all_trans['is_fraud'].map({1:'Fraud',0:'Non-Fraud'})
# normal = all_trans[df['is_fraud']==0]
# fraud = all_trans[df['is_fraud'] == 1]

# def stats_by_class(variable):
#   stat_grid = all_trans.groupby('class')[variable].agg([np.min,np.max,np.mean,np.median])
#   stat_grid = stat_grid.transpose().round(2)
#   return stat_grid

# def plot_box(data, x,y,title,width = 10, height = 7):
#   plt.figure(figsize = [width,height])
#   sns.boxplot(data = data, x=x,y=y)
#   plt.title(title);

# plot_box(all_trans,'class','amt','Distribution of Amount vs Class'); # calling the boxplot function

# # stats_by_class('amt') # calling the stats_by class function

In [ ]:
all_trans = df.copy()
all_trans['class'] = all_trans['isfraud'].map({1:'Fraud',0:'Non-Fraud'})
normal = all_trans[df['isfraud']==0]
fraud = all_trans[df['isfraud'] == 1]

def stats_by_class(variable):
  stat_grid = all_trans.groupby('class')[variable].agg([np.min,np.max,np.mean,np.median])
  stat_grid = stat_grid.transpose().round(2)
  return stat_grid

def plot_box(data, x,y,title,width = 10, height = 7):
  plt.figure(figsize = [width,height])
  sns.boxplot(data = data, x=x,y=y)
  plt.title(title)

#stats_by_class('amt') # calling the stats_by class function

plot_box(all_trans,'class','amt','Distribution of Amount vs Class'); # calling the boxplot function

fig = plt.subplots(figsize= (15,10))
plots = []
plots.append(sns.histplot(df[(df.isfraud == 0) & (df.amt <= 2000)].amt, bins=50, ax=plt.subplot(235)))
plots.append(sns.histplot(df[(df.isfraud ==1) & (df.amt <= 1500)].amt, bins =50, ax=plt.subplot(236)))

plots[0].set_title('Non-Fraud Amt Dist')
plots[1].set_title('Fraud Amt Dist')
plots[0].set_xlabel('Transaction Amount')
plots[1].set_xlabel('Transaction Amount')
plots[0].set_ylabel('Number of Transactions')
plots[1].set_ylabel('Number of Transactions')
plt.show()

stats_by_class('amt') # calling the stats_by class function

In [ ]:
ax = sns.histplot(x='amt', data=df[df.amt <= 2000], hue='isfraud',stat = 'percent',multiple= 'dodge',common_norm= False, bins = 25)
ax.set_ylabel('Percentage in Each Type')
ax.set_xlabel('Transaction amount in USD')
plt.legend(title='Type', labels = ['Fraud','Non-Fraud'])

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# def calculate_ratio(df, bin_edges):
#     # Create a new column 'amount_bin' based on transaction amount bins
#     df['amount_bin'] = pd.cut(df['amt'], bins=bin_edges)

#     # Group the DataFrame by 'amount_bin' and calculate counts of fraud and non-fraud transactions
#     grouped = df.groupby('amount_bin')['is_fraud'].value_counts(normalize=True).unstack().reset_index()

#     grouped[0] /= grouped[0].sum()
#     grouped[1] /= grouped[1].sum()

#     # Calculate the fraud-to-non-fraud ratio for each amount bin
#     grouped['fraud_ratio'] = grouped[1] / grouped[0]

#     return grouped

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_ratio(df, bin_edges):
    # Create a new column 'amount_bin' based on transaction amount bins
    df['amount_bin'] = pd.cut(df['amt'], bins=bin_edges)

    # Group the DataFrame by 'amount_bin' and calculate counts of fraud and non-fraud transactions
    grouped = df.groupby('amount_bin')['isfraud'].value_counts(normalize=True).unstack().reset_index()

    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the fraud-to-non-fraud ratio for each amount bin
    grouped['fraud_ratio'] = grouped[1] / grouped[0]

    return grouped


# Define the bin edges for transaction amounts
bin_edges = [0, 500, 1000, 2000]
bin_labels = ['0-499','500-999','1000-1999']
# Calculate the fraud-to-non-fraud ratio for transaction amounts
ratio_df = calculate_ratio(df, bin_edges)

# Convert the amount_bin intervals to strings
ratio_df['amount_bin'] = ratio_df['amount_bin'].astype(str)

# Sort the ratio_df DataFrame by fraud_ratio in ascending order
ratio_df_sorted = ratio_df.sort_values('fraud_ratio')

print (ratio_df_sorted)
# Plot all the groups on the bar chart in ascending order
plt.figure(figsize=(10, 5))
plt.bar(ratio_df_sorted['amount_bin'], ratio_df_sorted['fraud_ratio'])
plt.xlabel('Transaction Amount ($)')
plt.ylabel('Fraud Ratio')
plt.title('Fraud Ratio by Transaction Amount')
plt.xticks(ratio_df_sorted['amount_bin'],bin_labels, rotation=45)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_ratio(df, bin_edges):
    # Create a new column 'amount_bin' based on transaction amount bins
    df['amount_bin'] = pd.cut(df['amt'], bins=bin_edges)

    # Group the DataFrame by 'amount_bin' and calculate normalized counts of fraud and non-fraud transactions
    grouped = df.groupby('amount_bin')['isfraud'].value_counts(normalize=True).unstack().reset_index()

    # Normalize both fraud and non-fraud counts
    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the normalized fraud-to-non-fraud ratio for each amount bin
    grouped['fraud_ratio'] = grouped[1] / grouped[0]

    # Calculate the normalized non-fraud-to-fraud ratio for each amount bin
    grouped['non_fraud_ratio'] = grouped[0] / grouped[1]

    return grouped

@ticker.FuncFormatter
def major_formatter(x, pos):
    label = str(-x) if x < 0 else str(x)
    return label

# Define the bin edges for transaction amounts
bin_edges = [0, 500, 1000, 2000]
bin_labels = ['0-499', '500-999', '1000-1999']
# Calculate the normalized fraud-to-non-fraud and non-fraud-to-fraud ratios for transaction amounts
ratio_df = calculate_ratio(df, bin_edges)

# Convert the amount_bin intervals to strings
ratio_df['amount_bin'] = ratio_df['amount_bin'].astype(str)

# Sort the ratio_df DataFrame by fraud_ratio in ascending order
ratio_df_sorted = ratio_df.sort_values('fraud_ratio')

# Normalize the non-fraud ratio in the same way as the fraud ratio
max_non_fraud_ratio = ratio_df_sorted['fraud_ratio'].max()
ratio_df_sorted['non_fraud_ratio'] /= max_non_fraud_ratio

# Plot both normalized fraud and non-fraud ratios on the same bar chart
fig, ax1 = plt.subplots(figsize=(10, 5))

# Primary y-axis for Fraud Ratio
fraud_bars = ax1.barh(ratio_df_sorted['amount_bin'], ratio_df_sorted['fraud_ratio'], label='Fraud', color='blue')
ax1.set_xlabel('Fraud Ratio')
ax1.xaxis.set_major_formatter(major_formatter)
ax1.set_ylabel('Transaction Amount ($)')
ax1.set_yticks(ratio_df_sorted['amount_bin'])
ax1.set_yticklabels(bin_labels)


# Secondary y-axis for Non-Fraud Ratio
ax2 = ax1.twiny()
non_fraud_bars = ax2.barh(ratio_df_sorted['amount_bin'], -ratio_df_sorted['non_fraud_ratio'], label='Non-Fraud Ratio', alpha=0.7, color='orange')  # Use negative values for non-fraud ratio
ax2.set_xlabel('Non-Fraud Ratio')
ax2.xaxis.tick_top()
ax2.xaxis.set_label_position('top')
ax2.xaxis.set_major_formatter(major_formatter)

# Display the ratios on the bars with adjusted positions and rotated text
for bar, fraud_ratio, non_fraud_ratio in zip(fraud_bars, ratio_df_sorted['fraud_ratio'], ratio_df_sorted['non_fraud_ratio']):
    ax1.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, f'{fraud_ratio:.2f}', va='center', color='blue', fontsize=10, rotation=90)

for bar, non_fraud_ratio in zip(non_fraud_bars, ratio_df_sorted['non_fraud_ratio']):
    ax2.text(bar.get_width() - 0.01, bar.get_y() + bar.get_height()/2, f'{non_fraud_ratio:.2f}', va='center', ha='right', color='black', fontsize=10, rotation=90)

# Add legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='best')

plt.title('Fraud and Non-Fraud Ratios by Transaction Amount')
plt.tight_layout()
plt.show()

In [ ]:
# transforming to datetime format
df['trans_hour']= df['transdatetranstime'].dt.hour
df['trans_day_of_week'] = df['transdatetranstime'].dt.day_name()
#df['trans_year_month'] = df['transdatetranstime'].dt.to_period('M')
df.head()
df.dtypes

In [ ]:
# def sum_30_day(unixtime, cc_num):
#     unixstamp = unixtime
#     minus30 = unixstamp - 2629743
#     ccnum = cc_num
#     sumtable = df.loc[(df["cc_num"] == ccnum) & (df['unix_time'] < unixstamp) & (df['unix_time'] > minus30)]
#     history30 = sumtable['amt'].sum()
#     return history30


# # running function and creating a new variable for it
# df['history_30'] = df.apply(lambda x: sum_30_day(x.unix_time, x.cc_num), axis=1)


# # measuring interaction effect with amt in new variable
# df['interaction_30'] = df['history_30'] / df['amt']

In [ ]:
# calculate the age of customer
df['dob'] = pd.to_datetime(df['dob'])
df['age'] = ((df['transdatetranstime'] - df['dob']) / np.timedelta64(1, 'Y')).round(1)
df.age

In [ ]:
# Bin the age feature using cut function
age_bins = [0, 30, 45, 60, 75, np.inf]
age_labels = ['< 30', '30-45', '46-60', '61-75', '> 75']
df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=False)

df['age_group'].head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker  # Import ticker module for formatting

def calculate_ratio(df, bin_edges, column):
    # Create a new column 'group' based on binning the specified column
    df['group'] = pd.cut(df[column], bins=bin_edges)

    # Group the DataFrame by 'group' and calculate normalized counts of fraud and non-fraud transactions
    grouped = df.groupby('group')['isfraud'].value_counts(normalize=True).unstack().reset_index()

    # Normalize both fraud and non-fraud counts
    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the normalized fraud-to-non-fraud ratio for each age group
    grouped['fraud_ratio'] = grouped[1] / grouped[0]
    # Calculate the normalized non-fraud-to-fraud ratio for each age group
    grouped['non_fraud_ratio'] = grouped[0] / grouped[1]

    return grouped

# Define the bin edges for age groups
age_bin_edges = [0, 30, 45, 60, 75, np.inf]
bin_labels = ['< 30', '30 - 44', '45 - 59', '60 - 74', '75+']

# Calculate the normalized fraud-to-non-fraud and non-fraud-to-fraud ratios for age groups
age_ratio_df = calculate_ratio(df, age_bin_edges, 'age')

# Convert the age group intervals to strings
age_ratio_df['age_group'] = age_ratio_df['group'].astype(str)

# Plot both normalized fraud and non-fraud ratios on the same bar chart
plt.figure(figsize=(10, 5))
fraud_bars = plt.barh(age_ratio_df['age_group'], age_ratio_df['fraud_ratio'], label='Fraud Ratio', color='blue')
non_fraud_bars = plt.barh(age_ratio_df['age_group'], -age_ratio_df['non_fraud_ratio'], label='Non-Fraud Ratio', alpha=0.7, color='orange')  # Use negative values for non-fraud ratio

for bar in fraud_bars:
    plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, f'{bar.get_width():.2f}', va='center', color='black', fontsize=10)

for bar in non_fraud_bars:
    plt.text(bar.get_width() - 0.01, bar.get_y() + bar.get_height()/2, f'{-bar.get_width():.2f}', va='center', ha='right', color='black', fontsize=10)

plt.xlabel('Ratio')
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(-x) if x < 0 else str(x)))  # Set x-axis formatter
plt.ylabel('Age Group')
plt.title('Fraud and Non-Fraud Ratios by Age Group')
plt.legend(loc='upper left')  # Adjust the legend position
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize = (10,5))
ax = sns.histplot(data = df, x = 'trans_hour', hue= 'isfraud', common_norm = False, stat = 'percent', multiple = 'dodge')
ax.set_xlabel('Time(Hour) in a Day')
ax.set_ylabel('Percentage')
plt.xticks(np.arange(0,24,1))
plt.legend(title = 'Type', labels = ['Fraud','Non-Fraud'])
plt.show()

In [ ]:
# Define a function to map hour values to time intervals
def map_time_of_day(hour):
    if 4 <= hour < 7:
        return 'Early Morning'
    elif 7 <= hour < 11:
        return 'Morning'
    elif 11 <= hour < 16:
        return 'Afternoon'
    elif 16 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

# Apply the mapping function to create the 'time_of_day' column
df['time_of_day'] = df['trans_hour'].apply(map_time_of_day)

# Plot the histogram with the new 'time_of_day' column
plt.figure(figsize=(10, 5))
ax = sns.histplot(data=df, x='time_of_day', hue='isfraud', common_norm=False, stat='percent', multiple='dodge')
ax.set_xlabel('Time of Day')
ax.set_ylabel('Percentage')
plt.xticks(rotation=45)
plt.legend(title='Type', labels=['Fraud', 'Non-Fraud'])
plt.show()

df.time_of_day

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_ratio(df):
    # Group the DataFrame by 'time_of_day' and calculate normalized counts of fraud and non-fraud transactions
    grouped = df.groupby('time_of_day')['isfraud'].value_counts(normalize=True).unstack().reset_index()

    # Normalize both fraud and non-fraud counts for time of day
    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the normalized non-fraud-to-fraud ratio for each time category
    grouped['non_fraud_ratio'] = grouped[0] / grouped[1]

    # Calculate the normalized fraud-to-non-fraud ratio for each time category
    grouped['fraud_ratio'] = grouped[1] / grouped[0]

    return grouped

# Calculate the normalized non-fraud-to-fraud and fraud-to-non-fraud ratios for time of day
ratio_df = calculate_ratio(df)

# Plot both the non-fraud-to-fraud ratio and fraud-to-non-fraud ratio by time of day in a rotated graph
plt.figure(figsize=(10, 5))
fraud_bars = plt.barh(ratio_df['time_of_day'], ratio_df['fraud_ratio'], label='Fraud Ratio', color='blue')
non_fraud_bars = plt.barh(ratio_df['time_of_day'], -ratio_df['non_fraud_ratio'], label='Non-Fraud Ratio', alpha=0.7, color='orange')  # Use negative values for non-fraud ratio
plt.xlabel('Ratio')

plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(-x) if x < 0 else str(x)))
plt.ylabel('Time of Day')
plt.title('Fraud and Non-fraud Ratios by Time of Day')

# Display the ratios on the right side of each bar
for bar in fraud_bars:
    plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, f'{bar.get_width():.2f}', va='center', color='black', fontsize=10)

for bar in non_fraud_bars:
    plt.text(bar.get_width() - 0.01, bar.get_y() + bar.get_height()/2, f'{-bar.get_width():.2f}', va='center', ha='right', color='black', fontsize=10)

plt.yticks(rotation=45)  # Rotate y-axis labels
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
ax = sns.histplot(data= df, x='gender', hue='isfraud', stat='percent', common_norm = False, multiple = 'dodge')
ax.set_xlabel('Credit Card Holder Gender')
ax.set_ylabel('Percentage')
plt.legend(title = 'Type', labels= ['Fraud','Not-Fraud'])
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_ratio(df):
    # Group the DataFrame by 'gender' and calculate the normalized counts of fraud and non-fraud transactions
    grouped = df.groupby('gender')['isfraud'].value_counts(normalize=True).unstack().reset_index()

    # Normalize both fraud and non-fraud counts for gender
    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the normalized non-fraud-to-fraud ratio for each gender
    grouped['non_fraud_ratio'] = grouped[0] / grouped[1]

    # Calculate the normalized fraud-to-non-fraud ratio for each gender
    grouped['fraud_ratio'] = grouped[1] / grouped[0]

    return grouped

# Calculate the normalized non-fraud-to-fraud and fraud-to-non-fraud ratios for gender
ratio_df = calculate_ratio(df)

# Plot both the non-fraud-to-fraud ratio and fraud-to-non-fraud ratio by gender in opposite directions
plt.figure(figsize=(10, 5))
fraud_bars = plt.barh(ratio_df['gender'], ratio_df['fraud_ratio'], label='Fraud Ratio', color='blue')
non_fraud_bars = plt.barh(ratio_df['gender'], -ratio_df['non_fraud_ratio'], label='Non-Fraud Ratio', alpha=0.7, color='orange')  # Use negative values for non-fraud ratio
plt.ylabel('Gender')
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(-x) if x < 0 else str(x)))
plt.xlabel('Ratio')
plt.title('Fraud and Non-fraud Ratios by Gender')
plt.legend()

# Display the ratio values on the bars
for bar in fraud_bars:
    plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, f'{bar.get_width():.2f}', va='center', color='black', fontsize=10)

for bar in non_fraud_bars:
    plt.text(bar.get_width() - 0.01, bar.get_y() + bar.get_height()/2, f'{-bar.get_width():.2f}', va='center', ha='right', color='black', fontsize=10)

plt.show()


In [ ]:
# Calculate the value counts of each category
category_counts = df['category'].value_counts(normalize=True)

# Plot the count plot
plt.figure(figsize=(10, 5))
plot = sns.countplot(x='category', data=df, hue='isfraud')
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Transaction Count by Category')
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'])
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_ratio(df, column):
    # Group the DataFrame by the specified column and calculate normalized counts of fraud and non-fraud transactions
    grouped = df.groupby(column)['isfraud'].value_counts(normalize=True).unstack().reset_index()

    # Normalize both fraud and non-fraud counts
    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the normalized fraud-to-non-fraud ratio for each group
    grouped['fraud_ratio'] = grouped[1] / grouped[0]
    # Calculate the normalized non-fraud-to-fraud ratio for each group
    grouped['non_fraud_ratio'] = grouped[0] / grouped[1]

    return grouped

# Calculate the normalized fraud-to-non-fraud and non-fraud-to-fraud ratios for merchant categories
category_ratio_df = calculate_ratio(df, 'category')

# Sort the DataFrame by 'category' in ascending order
category_ratio_df_sorted = category_ratio_df.sort_values('category')

# Plot both normalized fraud and non-fraud ratios on the same bar chart
plt.figure(figsize=(10, 5))
fraud_bars = plt.barh(category_ratio_df_sorted['category'], category_ratio_df_sorted['fraud_ratio'], label='Fraud Ratio', color='blue')
non_fraud_bars = plt.barh(category_ratio_df_sorted['category'], -category_ratio_df_sorted['non_fraud_ratio'], label='Non-Fraud Ratio', alpha=0.7, color='orange')  # Use negative values for non-fraud ratio

for bar in fraud_bars:
    plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, f'{bar.get_width():.2f}', va='center', color='black', fontsize=10)

for bar in non_fraud_bars:
    plt.text(bar.get_width() - 0.01, bar.get_y() + bar.get_height()/2, f'{-bar.get_width():.2f}', va='center', ha='right', color='black', fontsize=10)

plt.xlabel('Ratio')
plt.ylabel('Category')
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(-x) if x < 0 else str(x)))
plt.title('Fraud and Non-Fraud Ratios by Category')
plt.legend(loc='lower right')  # Adjust the legend position
plt.tight_layout()
plt.show()


In [67]:
# remove first name, last name, data of birth and transaction date/time
df.drop(['first','last','dob','transdatetranstime','transdate'], axis = 1, inplace= True)

In [68]:
# storing a copy
df_org = df.copy()

In [ ]:
100*df.isfraud.value_counts(normalize=True)

In [ ]:
# df_sorted.nunique()

In [ ]:
# df2['distance'] = np.sqrt((df['lat']-df['merch_lat'])**2 + (df['long']-df['merch_long'])**2)

In [ ]:
# df = df_sorted.copy()
# df.info()

In [70]:
category_onehot = pd.get_dummies(df.category, prefix = 'category',drop_first = True)
gender_onehot = pd.get_dummies(df.gender, prefix = 'gender', drop_first = True)
day_of_week_onehot = pd.get_dummies(df.trans_day_of_week, prefix = 'day', drop_first = True)
time_of_day_onehot = pd.get_dummies(df.time_of_day, prefix = 'time_of_day', drop_first = True)
#trans_year_month_onehot = pd.get_dummies(df.trans_year_month, prefix = 'year', drop_first = True)
age_group_onehot = pd.get_dummies(df.age_group , prefix = 'age', drop_first = True)

In [72]:
df2 = pd.concat([df, category_onehot,gender_onehot,day_of_week_onehot,time_of_day_onehot,age_group_onehot],axis = 1)

In [73]:
df2.drop(['ccnum', 'recnum','transnum','amount_bin','merchant','street','city','state','job','category','gender','trans_day_of_week','age','lat','long','citypop','unixtime','merchlat','merchlong','trans_hour','age_group','group','time_of_day'],axis = 1, inplace = True)


In [ ]:
print(df2.shape)
df2.columns

In [ ]:
# df1.drop(['lat','long','city_pop','merch_lat','merch_long'],axis = 1, inplace = True)
# print(df1.columns)

In [ ]:
# fig, ax = plt.subplots(figsize=(20,10))
# sns.heatmap(df1.corr(),cmap = 'coolwarm').set_title('Correlation')

In [ ]:
# fig = plt.figure(figsize = (18,9))
# sns.heatmap(df2.corr(), cmap = 'coolwarm', annot = False)
# plt.show()

In [75]:
non_fraud_count, fraud_count = df2.isfraud.value_counts()

In [ ]:
df_num = df2.select_dtypes(include = 'number')
#df_num.drop(['index'], axis = 1, inplace = True)
df_num.columns

In [77]:
# Data Standardiztion
from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()
# Scales all variables
df_scaled = rob_scaler.fit_transform(df_num)
df_num = pd.DataFrame(df_scaled, columns=df_num.columns)

In [ ]:
from sklearn.model_selection import StratifiedKFold

print('No Frauds', round(df_num['isfraud'].value_counts()[0] / len(df_num) * 100, 2), '% of the dataset')
print('Frauds', round(df_num['isfraud'].value_counts()[1] / len(df_num) * 100, 2), '% of the dataset')

X = df_num.drop('isfraud', axis=1)
y = df_num['isfraud']

sss = StratifiedKFold(n_splits=5, random_state= None , shuffle= True)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]


# Check the Distribution of the labels

# Turn into arrays
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)



In [ ]:
# Since our classes are highly skewed, we should make them equivalent in order to have a balanced distribution of the classes.

# Let's shuffle the data before creating the subsamples
df_num = df_num.sample(frac=1)

# Amount of fraud cases (453 rows).
fraud_df = df_num.loc[df_num['isfraud'] == 1]
non_fraud_df = df_num.loc[df_num['isfraud'] == 0][:453]  # Update the number to match your fraud cases count

balanced_df = pd.concat([fraud_df, non_fraud_df])

# Shuffle dataframe rows
new_df = balanced_df.sample(frac=1, random_state=42)

new_df.head()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

print('Distribution of the Classes in the subsample dataset')
print(new_df['isfraud'].value_counts() / len(new_df))

colors = ["#0101DF", "#DF0101"]  # Blue for non-fraud, Red for fraud

plt.figure(figsize=(7, 4))  # Adjust the figure size if needed
sns.countplot(x='isfraud', data=new_df, palette=colors)
plt.title('Equally Distributed Classes', fontsize=14)
plt.show()


In [81]:
## Feature Selection using LASSO ##
X_cols = new_df.columns.drop('isfraud')
Y_cols = 'isfraud'

In [ ]:
# Implementing LASSO regression onto dataset
from sklearn.linear_model import LassoCV
X = pd.DataFrame(new_df[X_cols], columns=X_cols) # feature matrix
y = new_df[Y_cols] # target variable

reg = LassoCV()
reg.fit(X,y)
print("Best alpha using built_in LassoCV = %f" %reg.alpha_)
print("Best score using built-in LassoCV = %f" %reg.score(X,y))

In [ ]:
coef = pd.Series(reg.coef_, index=X.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " + str(sum(coef == 0)) + " variables")
imp_coef = coef[coef != 0].sort_values()

import matplotlib.pyplot as plt
plt.figure(figsize=(8.0, 10.0))
imp_coef.plot(kind="barh")
plt.title("Important Variables Selected by Lasso model")
plt.xlabel("Coefficient Value")
plt.ylabel("Variable")
plt.show()

picked_variables = X.columns[coef != 0]
print("Variables picked by Lasso:", picked_variables)


In [84]:
selected_columns = ['ccnum_freq3_transdate', 'ccnum_freq30_transdate',
       'ccnum_0_dayfreq_div_14_dayfreq_velchange',
       'ccnum_0_dayfreq_div_30_dayfreq_velchange',
       'merchant_0_dayfreq_div_30_dayfreq_velchange',
       'card-merchant_daysSince', 'ccnum_totalamount1_amt2',
       'merchant_totalamount7_amt2', 'merchant_totalamount14_amt2',
       'merchant_totalamount30_amt2', 'ccnum_median0_amt2',
       'ccnum_median1_amt2', 'ccnum_median30_amt2', 'merchant_median1_amt2',
       'merchant_median7_amt2', 'merchant_median30_amt2',
       'merchant_mean0_amt2', 'merchant_mean1_amt2', 'ccnum_max1_amt2',
       'ccnum_max3_amt2', 'ccnum_max30_amt2', 'merchant_max30_amt2',
       'ccnum_median_1_actual', 'ccnum_median_3_actual',
       'ccnum_median_7_actual', 'ccnum_median_14_actual',
       'ccnum_median_30_actual', 'merchant_median_3_actual',
       'merchant_median_7_actual', 'merchant_median_14_actual',
       'merchant_median_30_actual', 'ccnum_mean_30_actual',
       'merchant_mean_3_actual', 'merchant_mean_30_actual',
       'ccnum_max_14_actual', 'merchant_max_3_actual', 'merchant_max_7_actual',
       'category_grocery_pos', 'category_shopping_net', 'day_Monday',
       'day_Wednesday', 'time_of_day_Night', 'age_61-75', 'isfraud']

df_filtered = df_num[selected_columns]

In [85]:
# Logistic Regression - imbalanced data
X = df_filtered.drop('isfraud', axis=1)
y = df_filtered['isfraud']

In [86]:
# Our data is already scaled we should split our training and test sets
from sklearn.model_selection import train_test_split

# Splitting Train-Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Define the hyperparameter grid
log_reg_params = {'C': [0.001, 0.01, 0.1, 1.0], 'max_iter': [2000], 'random_state': [13]}

# Create a cross-validation strategy (Stratified K-Fold)
cv = StratifiedKFold(n_splits=5, shuffle=False)

# Initialize GridSearchCV with Logistic Regression, hyperparameter grid, and cross-validation
grid_log_reg = GridSearchCV(LogisticRegression(random_state=13), log_reg_params, cv=cv, scoring='accuracy')

# Fit the grid search to your training data
grid_log_reg.fit(X_train, y_train)

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
classifier = XGBClassifier()

In [ ]:
from datetime import datetime

start_time = timer(None) # timing starts from this point for "start_time" variable
classifier.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

In [ ]:
xgb = XGBClassifier(random_state = 13)

params = {
    "learning_rate": [0.1,1,3],
    "max_depth":[6,7,8],
    "subsample": [0.5, 0.7, 1.0],
    "colsample_bytree": [0.5, 0.8, 1.0],
    "n_estimators" : [500,600,1000]
    }

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

xgb_grid = GridSearchCV(estimator=xgb, param_grid=params, scoring="recall", cv=cv)
xgb_grid.fit(X, y)

In [ ]:
predictionXGB = xgb.predict(X_test)
print(confusion_matrix(y_test,predictionXGB))
print(classification_report(y_test,predictionXGB))

In [ ]:
# Using Optuna for hyperparameter tuning
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 15),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.6, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01,1.0),
        'n_estimators': trial.suggest_int('n_estimators', 600,1000,100),
        'eval_metric': 'aucpr'
    }

    # Fit the model
    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train, y_train)

    # Make predictions
    y_pred = optuna_model.predict(X_test)

    # Evaluate predictions
    recall = recall_score(y_test, y_pred)
    return recall

In [ ]:
!pip3 install optuna

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import optuna

In [ ]:
study = optuna.create_study(direction='maximize')

In [ ]:
study.optimize(objective, n_trials=20)

In [ ]:
trial = study.best_trial

In [ ]:
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')

print('  Value: {:.2f}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {:.3f}'.format(key, value))

In [ ]:
params = trial.params

In [ ]:
model = XGBClassifier(**params)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

xgb_matrix = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(xgb_matrix), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = model.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="XGBoost, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    model, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall curve with Class weights")

In [ ]:
# XGboost Random Undersampling
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 15),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.6, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01,1.0),
        'n_estimators': trial.suggest_int('n_estimators', 600,1000,100),
        'eval_metric': 'aucpr'
    }

    # Apply Random Undersampling to the training data
    sampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
    X_train_rus, y_train_rus = sampler.fit_resample(X_train, y_train)

    # Fit the model on the resampled data
    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train_rus, y_train_rus)

    # Make predictions
    y_pred = optuna_model.predict(X_test)

    # Evaluate predictions
    recall = recall_score(y_test, y_pred)
    return recall

In [ ]:
# Create an Optuna study
study = optuna.create_study(direction='maximize')

# Optimize hyperparameters
study.optimize(objective, n_trials=20)

# Get the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)

In [ ]:
trial = study.best_trial

In [ ]:
params_rus = trial.params

In [ ]:
XGB_under = XGBClassifier(**params_rus)
XGB_under.fit(X_train, y_train)

In [ ]:
y_pred = XGB_under.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

matrix_xgb_under = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(matrix_xgb_under), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = XGB_under.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="XGBoost, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    XGB_under, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall curve with Class weights")

In [ ]:
# XGboost Random Oversampling
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 15),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.6, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01,1.0),
        'n_estimators': trial.suggest_int('n_estimators', 600,1000,100),
        'eval_metric': 'aucpr'
    }

    # Apply Random Oversampling to the training data
    sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
    X_train_ros, y_train_ros = sampler.fit_resample(X_train, y_train)

    # Fit the model on the resampled data
    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train_ros, y_train_ros)

    # Make predictions
    y_pred = optuna_model.predict(X_test)

    # Evaluate predictions
    recall = recall_score(y_test, y_pred)
    return recall

In [ ]:
# Create an Optuna study
study = optuna.create_study(direction='maximize')

# Optimize hyperparameters
study.optimize(objective, n_trials=20)

best_params = study.best_params
formatted_params = {key: f"{value:.3f}" for key, value in best_params.items()}

print("Best Hyperparameters:")
for key, value in formatted_params.items():
    print(f"  {key}: {value}")



In [ ]:
trial = study.best_trial

In [ ]:
params_ros = trial.params

In [ ]:
XGB_over = XGBClassifier(**params_ros)
XGB_over.fit(X_train, y_train)

In [ ]:
predict_xgb_over = XGB_over.predict(X_test)
print(confusion_matrix(y_test,predict_xgb_over))
print(classification_report(y_test,predict_xgb_over))

In [ ]:
%matplotlib inline

cnf_matrix = confusion_matrix(y_test,predict_xgb_over)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = XGB_over.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="Logistic Regression, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    XGB_over, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall curve with Class weights")

In [ ]:
# XGboost SMOTE
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 15),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.6, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01,1.0),
        'n_estimators': trial.suggest_int('n_estimators', 600,1000,100),
        'eval_metric': 'aucpr'
    }

    # Apply Random Oversampling to the training data
    sampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_smote, y_train_smote = sampler.fit_resample(X_train, y_train)

    # Fit the model on the resampled data
    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train_smote, y_train_smote)

    # Make predictions
    y_pred = optuna_model.predict(X_test)

    # Evaluate predictions
    recall = recall_score(y_test, y_pred)
    return recall

In [ ]:
# Create an Optuna study
study = optuna.create_study(direction='maximize')

# Optimize hyperparameters
study.optimize(objective, n_trials=10)

best_params = study.best_params
formatted_params = {key: f"{value:.3f}" for key, value in best_params.items()}

print("Best Hyperparameters:")
for key, value in formatted_params.items():
    print(f"  {key}: {value}")

In [ ]:
trial = study.best_trial

In [ ]:
params_smote = trial.params

In [ ]:
XGB_smote = XGBClassifier(**params_smote)
XGB_smote.fit(X_train, y_train)

In [ ]:
y_pred = XGB_smote.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

matrix_xgb_under = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(matrix_xgb_under), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = XGB_smote.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="XGBoost, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    XGB_smote, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall curve with Class weights")

In [ ]:
# Turn the values into an array for feeding the classification algorithms.
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
# lr = LogisticRegression(C = 1.0, random_state=42, max_iter = 1500)
# model = lr.fit(X_train, y_train)
# y_train_pred = model.predict(X_train)
# print('y_train_pred: ',y_train_pred)
# y_test_pred = model.predict(X_test)
# print('y_test_pred: ', y_test_pred)

In [ ]:
# #evaluating the model
# model_name = 'Logistic Regression - without balancing'
# train_score = model.score(X_train,y_train)
# test_score = model.score(X_test,y_test)

# acc_score = accuracy_score(y_test,y_test_pred)
# f_score = f1_score(y_test, y_test_pred)
# precision = precision_score(y_test, y_test_pred)
# recall = metrics.recall_score(y_test,y_test_pred)
# #creating a dataframe to compare the performance of different models
# model_eval_data = [[model_name, train_score, test_score, acc_score, f_score, precision, recall]]
# evaluate_df = pd.DataFrame(model_eval_data, columns=['Model Name', 'Training Score', 'Testing Score', 'Accuracy',
#                                           'F1 Score', 'Precision', 'Recall'])
# evaluate_df

In [ ]:
# report = classification_report(y_test, y_test_pred)

# # Print the classification report
# print(report)

In [ ]:
# from sklearn.model_selection import GridSearchCV


# # Logistic Regression
# log_reg_params = {'C': [0.001, 0.01, 0.1, 1.0],'max_iter' : [2000]}


# grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
# grid_log_reg.fit(X_train, y_train)
# # We automatically get the logistic regression with the best parameters.
# log_reg = grid_log_reg.best_estimator_

# print("Best C value for Logistic Regression: ", grid_log_reg.best_params_['C'])

In [ ]:
# Logistic Regression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Define the hyperparameter grid
log_reg_params = { 'C': [0.001, 0.01, 0.1, 1.0], 'max_iter' :[2000], 'random_state':[13]}

# Create a cross-validation strategy (Stratified K-Fold)
cv = StratifiedKFold(n_splits=5, shuffle=False)

# Initialize GridSearchCV with Logistic Regression, hyperparameter grid, and cross-validation
grid_log_reg = GridSearchCV(LogisticRegression(random_state = 13), log_reg_params, cv=cv, scoring = 'recall')

# Fit the grid search to your training data
grid_log_reg.fit(X_train, y_train)

# Print the best hyperparameters
print("Best hyperparameters for Logistic Regression:", grid_log_reg.best_params_['C'])

# Get the best estimator
best_log_reg = grid_log_reg.best_estimator_

In [ ]:
predictionLR = grid_log_reg.predict(X_test)
print(confusion_matrix(y_test,predictionLR))
print(classification_report(y_test,predictionLR))

In [ ]:
%matplotlib inline

cnf_matrix = confusion_matrix(y_test,predictionLR)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = best_log_reg.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="Logistic Regression, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    best_log_reg, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall curve with Class weights")

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=False)

In [ ]:
# Logistic Regression - Undersampling
rus = RandomUnderSampler(random_state = 42)
X_rus, y_rus = rus.fit_resample(X_train, y_train)

In [ ]:
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression

logistic_under_pipeline = make_pipeline(RandomUnderSampler(random_state=42),
                                        LogisticRegression(random_state=13, max_iter=1500))

In [ ]:
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Define the hyperparameter grid, specifying 'logisticregression__' for the C parameter
new_params = {
    'logisticregression__C': [0.001, 0.01, 0.1,1.0]
}

# Create a GridSearchCV instance, specifying the pipeline, parameter grid, cv, scoring, etc.
grid_under_logistic = GridSearchCV(
    logistic_under_pipeline,  # Your pipeline with Logistic Regression
    param_grid=new_params,  # The hyperparameter grid
    cv= cv ,  # Your cross-validation strategy (StratifiedKFold)
    scoring='recall',  # The scoring metric you want to optimize for
    return_train_score=True
)

# Fit the grid search to your data
grid_under_logistic.fit(X_train, y_train)


In [ ]:
print('Best parameters:', grid_under_logistic.best_params_)
print('Best score:', grid_under_logistic.best_score_)

In [ ]:
y_pred = grid_under_logistic.best_estimator_.named_steps['logisticregression'].predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

matrix_under = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(matrix_under), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = grid_under_logistic.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="Logistic Regression, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    grid_under_logistic, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall curve with Class weights")

In [ ]:
# Logistic Regression - Oversampling
ros = RandomOverSampler(random_state = 42)
X_ros, y_ros = ros.fit_resample(X_train, y_train)

In [ ]:
import numpy as np

# Calculate the unique values and their counts
unique_values, counts = np.unique(y_ros, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f'Class {value}: {count} samples')


In [ ]:
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression

logistic_over_pipeline = make_pipeline(RandomOverSampler(random_state=42),
                                        LogisticRegression(random_state=13, max_iter=1500))

score2 = cross_val_score(logistic_over_pipeline, X_train, y_train, scoring='recall', cv= cv)
print("Cross Validation Recall Scores are: {}".format(score2))
print("Average Cross Validation Recall score: {}".format(score2.mean()))

In [ ]:
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Define the hyperparameter grid, specifying 'logisticregression__' for the C parameter
new_params = {
    'logisticregression__C': [0.001, 0.01, 0.1,1.0]
}

# Create a GridSearchCV instance, specifying the pipeline, parameter grid, cv, scoring, etc.
grid_over_logistic = GridSearchCV(
    logistic_over_pipeline,  # Your pipeline with Logistic Regression
    param_grid=new_params,  # The hyperparameter grid
    cv= cv ,  # Your cross-validation strategy (StratifiedKFold)
    scoring='recall',  # The scoring metric you want to optimize for
    return_train_score=True
)

# Fit the grid search to your data
grid_over_logistic.fit(X_train, y_train)


In [ ]:
print('Best parameters:', grid_over_logistic.best_params_)
print('Best score:', grid_over_logistic.best_score_)

In [ ]:
y_pred = grid_over_logistic.best_estimator_.named_steps['logisticregression'].predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

matrix_over = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(matrix_over), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = grid_over_logistic.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="Logistic Regression, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    grid_over_logistic, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall curve with Class weights")

In [ ]:
## Logistic Regression - SMOTE #
from imblearn.over_sampling import SMOTE

smote_pipeline = make_pipeline(SMOTE(random_state=42),
                              LogisticRegression(max_iter = 2000, random_state=13))
score2 = cross_val_score(smote_pipeline, X_train, y_train, cv= cv)
print("Cross Validation Recall Scores are: {}".format(score2))
print("Average Cross Validation Recall score: {}".format(score2.mean()))

In [ ]:
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Define the hyperparameter grid, specifying 'logisticregression__' for the C parameter
new_params = {
    'logisticregression__C': [0.001, 0.01, 0.1,1.0]
}

# Create a GridSearchCV instance, specifying the pipeline, parameter grid, cv, scoring, etc.
grid_smote_logistic = GridSearchCV(
    smote_pipeline,  # Your pipeline with Logistic Regression
    param_grid=new_params,  # The hyperparameter grid
    cv= cv ,  # Your cross-validation strategy (StratifiedKFold)
      # The scoring metric you want to optimize for
    return_train_score=True
)

# Fit the grid search to your data
grid_smote_logistic.fit(X_train, y_train)


In [ ]:
print('Best parameters:', grid_smote_logistic.best_params_)
print('Best score:', grid_smote_logistic.best_score_)

In [ ]:
y_pred = grid_smote_logistic.best_estimator_.named_steps['logisticregression'].predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

matrix_smote = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(matrix_smote), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = grid_smote_logistic.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="Logistic Regression, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    grid_smote_logistic, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall curve with Class weights")

In [ ]:
## Random Forest - imbalanced dataset #
kf = StratifiedKFold(n_splits=5, shuffle=False)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=13)

In [ ]:
score = cross_val_score(rf, X_train, y_train, cv=kf)
print("Cross Validation Recall scores are: {}".format(score))
print("Average Cross Validation Recall score: {}".format(score.mean()))

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [50, 100, 200],
    'random_state': [13]
}

grid_rf = GridSearchCV(rf, param_grid=params, cv=kf,
                          scoring='recall').fit(X_train, y_train)

In [ ]:
print('Best parameters:', grid_rf.best_params_)
print('Best score:', grid_rf.best_score_)

In [ ]:
y_pred = grid_rf.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

rf_matrix = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(rf_matrix), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = grid_rf.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="Random Forest, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    grid_rf, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall curve with Class weights")

In [ ]:
## Random Forest - Oversampling
from imblearn.over_sampling import RandomOverSampler

rf_over_pipeline = make_pipeline(RandomOverSampler(random_state=42),
                              RandomForestClassifier(n_estimators = 100, random_state=13))

In [ ]:
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Define the hyperparameter grid for Random Forest
params = {
    'randomforestclassifier__n_estimators': [50, 100, 200],
    'randomforestclassifier__random_state': [13]
}

# Create a GridSearchCV instance, specifying the pipeline, parameter grid, cv, scoring, etc.
grid_over_rf = GridSearchCV(
    rf_over_pipeline,  # Your pipeline with Random Forest
    param_grid=params,  # The hyperparameter grid
    cv=cv,  # Your cross-validation strategy (StratifiedKFold)
    scoring='recall',  # The scoring metric you want to optimize for
    return_train_score=True
)

# Fit the grid search to your data
grid_over_rf.fit(X_train, y_train)


In [ ]:
print('Best parameters:', grid_over_rf.best_params_)
print('Best score:', grid_over_rf.best_score_)

In [ ]:
y_pred = grid_over_rf.best_estimator_.named_steps['randomforestclassifier'].predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

rf_over = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(rf_over), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = grid_over_rf.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="Random Forest, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    grid_over_rf, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall Curve with Class weights")

In [ ]:
## Random Forst Undersampling
from imblearn.pipeline import Pipeline, make_pipeline

rf_under_pipeline = make_pipeline(RandomUnderSampler(random_state=42),
                              RandomForestClassifier(n_estimators=100, random_state=13))


In [ ]:
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Define the hyperparameter grid for Random Forest
params = {
    'randomforestclassifier__n_estimators': [50, 100, 200],
    'randomforestclassifier__random_state': [13]
}

# Create a GridSearchCV instance, specifying the pipeline, parameter grid, cv, scoring, etc.
grid_under_rf = GridSearchCV(
    rf_under_pipeline,  # Your pipeline with Random Forest
    param_grid=params,  # The hyperparameter grid
    cv=cv,  # Your cross-validation strategy (StratifiedKFold)
    scoring='recall',  # The scoring metric you want to optimize for
    return_train_score=True
)

# Fit the grid search to your data
grid_under_rf.fit(X_train, y_train)


In [ ]:
print('Best parameters:', grid_under_rf.best_params_)
print('Best score:', grid_under_rf.best_score_)

In [ ]:
y_pred = grid_under_rf.best_estimator_.named_steps['randomforestclassifier'].predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

rf_under = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(rf_under), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = grid_under_rf.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="Random Forest, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    grid_under_rf, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall Curve with Class weights")

In [ ]:
## Random Forest SMOTE
from imblearn.over_sampling import SMOTE

rf_smote_pipeline = make_pipeline(SMOTE(random_state=42),
                              RandomForestClassifier(n_estimators=100, random_state=13))


In [ ]:
# Define the hyperparameter grid for Random Forest
params = {
    'randomforestclassifier__n_estimators': [50, 100, 200],
    'randomforestclassifier__random_state': [13]
}

# Create a GridSearchCV instance, specifying the pipeline, parameter grid, cv, scoring, etc.
grid_smote_rf = GridSearchCV(
    rf_smote_pipeline,  # Your pipeline with Random Forest
    param_grid=params,  # The hyperparameter grid
    cv=cv,  # Your cross-validation strategy (StratifiedKFold)
    scoring='recall',  # The scoring metric you want to optimize for
    return_train_score=True
)

# Fit the grid search to your data
grid_smote_rf.fit(X_train, y_train)

In [ ]:
print('Best parameters:', grid_smote_rf.best_params_)
print('Best score:', grid_smote_rf.best_score_)

In [ ]:
y_pred = grid_smote_rf.best_estimator_.named_steps['randomforestclassifier'].predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

rf_smote = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(rf_smote), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = grid_smote_rf.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="Random Forest, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    grid_smote_rf, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall Curve with Class weights")

In [ ]:
## XGBoost - imbalanced dataset
def xgboost_search(X, y, search_verbose=1):
    params = {
    "gamma":[0.5, 1, 1.5, 2, 5],
    "max_depth":[3,4,5,6],
    "min_child_weight": [100],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "learning_rate": [0.1, 0.01, 0.001]
    }
    xgb = XGBClassifier(objective="binary:logistic", eval_metric="auc", use_label_encoder=False)

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1234)

    grid_search = GridSearchCV(estimator=xgb, param_grid=params, scoring="roc_auc", n_jobs=1, cv=skf.split(X,y), verbose=search_verbose)

    grid_search.fit(X, y)

    print("Best estimator: ")
    print(grid_search.best_estimator_)
    print("Parameters: ", grid_search.best_params_)
    print("Highest AUC: %.2f" % grid_search.best_score_)

    return grid_search.best_params_

In [ ]:
import random
rows = random.sample(np.arange(0,len(X_train.index)).tolist(), 5000)

In [ ]:
## Random Forest - imbalanced dataset #
kf = StratifiedKFold(n_splits=5, shuffle=False)

In [ ]:
xgb = XGBClassifier(random_state=13)

In [ ]:
score = cross_val_score(xgb, X_train, y_train, cv=kf)
print("Cross Validation Recall scores are: {}".format(score))
print("Average Cross Validation Recall score: {}".format(score.mean()))

In [ ]:
xgb.fit(X_train, y_train.ravel())

In [ ]:
from sklearn.model_selection import GridSearchCV

params={
 "learning_rate"    : [0.05,0.1, 0.3] ,
 "max_depth"        : [ 12, 15]

}
grid_xgb= GridSearchCV(estimator = xgb,param_grid=params,scoring='f1',n_jobs=-1,cv=kf)

# grid_rf = GridSearchCV(rf, param_grid=params, cv=kf,
#                           scoring='recall').fit(X_train, y_train)

In [ ]:
grid_xgb.fit(X_train,y_train.ravel())

In [ ]:
print('Best parameters:', grid_xgb.best_params_)
print('Best score:', grid_xgb.best_score_)

In [ ]:
y_pred = grid_xgb.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
%matplotlib inline

xgb_matrix = confusion_matrix(y_test,y_pred)

class_names=['Non-Fraud', 'Fraud'] # name  of classes
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(pd.DataFrame(xgb_matrix), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("bottom")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = [0.5, 1.5]
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Get predicted probabilities from the best Logistic Regression model
y_proba = grid_xgb.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_proba)

# Calculate the AUC
auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label="Random Forest, AUC={:.3f}".format(auc))
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

# Display AUC score at the bottom left of the curve
plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size': 13}, loc='lower right')

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    grid_rf, X_test, y_test, name="Average precision")
_ = display.ax_.set_title("Precision-Recall curve with Class weights")

In [ ]:
#innput-output split
X = df_num.drop(['isfraud'],axis=1)
y = df_num.isfraud

In [ ]:
#train-test split using stratified K fold
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X,y)

for train_index, test_index in skf.split(X,y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

y_train.value_counts()

In [ ]:
lr = LogisticRegression(random_state=42, solver = 'liblinear')
model = lr.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('y_train_pred: ',y_train_pred)
y_test_pred = model.predict(X_test)
print('y_test_pred: ', y_test_pred)

In [ ]:
#evaluating the model
model_name = 'Logistic Regression - without balancing'
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)

acc_score = accuracy_score(y_test,y_test_pred)
f_score = f1_score(y_test, y_test_pred, average='weighted')
precision = precision_score(y_test, y_test_pred)
recall = metrics.recall_score(y_test,y_test_pred)
#creating a dataframe to compare the performance of different models
model_eval_data = [[model_name, train_score, test_score, acc_score, f_score, precision, recall]]
evaluate_df = pd.DataFrame(model_eval_data, columns=['Model Name', 'Training Score', 'Testing Score', 'Accuracy',
                                          'F1 Score', 'Precision', 'Recall'])
evaluate_df

In [ ]:
#renaming X_test and y_test variables
test_input = X_test.copy()
test_output = y_test.copy()

test_output.value_counts(normalize=True)

In [ ]:
#storing all fraud transactions
fraud_trans = df_num[df_num['isfraud'] == 1]
non_fraud_trans = df_num[df_num['isfraud'] == 0]

print('fraud data shape: ', fraud_trans.shape)
print('non fraud data shape: ', non_fraud_trans.shape)

#printing fraud data percentage
print('Fraud Data percentage: ', 100*(len(fraud_trans)/len(non_fraud_trans)))

In [ ]:
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X_train,y_train)

y_rus.value_counts()

In [ ]:
# Train-test Split
X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled = train_test_split(X_rus, y_rus, test_size=0.3, random_state=42, stratify=y_rus)
y_train_undersampled.value_counts()

In [ ]:
#creating a baseline linear model
lr = LogisticRegression(random_state=42, max_iter = 1000)
#model
model = lr.fit(X_train_undersampled, y_train_undersampled)
y_train_pred = model.predict(X_train)
y_train_pred

In [ ]:
#predicting on imbalanced test data
test_pred=model.predict(test_input)

In [ ]:
#printing the classification report of the model
print(classification_report(test_output,test_pred))

In [ ]:
#evaluating the model
model_name = 'Logistic Regression - Random Under Sampling'
train_score = model.score(X_train_undersampled,y_train_undersampled)
test_score = model.score(X_test_undersampled,y_test_undersampled)

acc_score = accuracy_score(test_output,test_pred)
f_score = f1_score(test_output, test_pred)
precision = precision_score(test_output, test_pred)
recall = metrics.recall_score(test_output,test_pred)
#adding claculations to dataframe
model_eval_data = [model_name, train_score, test_score, acc_score, f_score, precision, recall]
model_eval_dict = {evaluate_df.columns[i]:model_eval_data[i] for i in range(len(model_eval_data))}
evaluate_df = evaluate_df.append(model_eval_dict, ignore_index=True)

evaluate_df

In [ ]:
#oversampling with imblearn
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X_train,y_train)

y_ros.value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)

In [ ]:
evaluate_df = pd.DataFrame(columns=['Model Name', 'Training Score', 'Testing Score', 'Accuracy', 'F1 Score', 'Precision', 'Recall'])

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from imblearn.under_sampling import RandomUnderSampler


for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize and train a Logistic Regression model
    lr = LogisticRegression(random_state=42, max_iter = 1000, solver = 'liblinear')
    model = lr.fit(X_train, y_train.values.ravel())

    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate the model
    model_name = 'Logistic Regression - without balancing'
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    acc_score = accuracy_score(y_test, y_test_pred)
    f_score = f1_score(y_test, y_test_pred, average='weighted')
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)

    # Store evaluation results in the dataframe
    model_eval_data = [model_name, train_score, test_score, acc_score, f_score, precision, recall]
    model_eval_dict = {evaluate_df.columns[i]: model_eval_data[i] for i in range(len(model_eval_data))}
    evaluate_df = evaluate_df.append(model_eval_dict, ignore_index=True)

In [ ]:
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X_train, y_train)

In [ ]:
# Train-test Split
X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled = train_test_split(X_rus, y_rus, test_size=0.3, random_state=42, stratify=y_rus)

# Create a new Logistic Regression model on the under-sampled data
lr_undersampled = LogisticRegression(random_state=42)
model_undersampled = lr_undersampled.fit(X_train_undersampled, y_train_undersampled)

# Predictions on under-sampled test data
test_pred_undersampled = model_undersampled.predict(X_test_undersampled)

# Print classification report for the under-sampled model
print(classification_report(y_test_undersampled, test_pred_undersampled))

In [ ]:
model_name = 'Logistic Regression - Random Under Sampling'
train_score_undersampled = model_undersampled.score(X_train_undersampled, y_train_undersampled)
test_score_undersampled = model_undersampled.score(X_test_undersampled, y_test_undersampled)

acc_score_undersampled = accuracy_score(y_test_undersampled, test_pred_undersampled)
f_score_undersampled = f1_score(y_test_undersampled, test_pred_undersampled, average='weighted')
precision_undersampled = precision_score(y_test_undersampled, test_pred_undersampled)
recall_undersampled = recall_score(y_test_undersampled, test_pred_undersampled)


In [ ]:
# Add under-sampled model evaluation results to the dataframe
model_eval_data_undersampled = [model_name, train_score_undersampled, test_score_undersampled, acc_score_undersampled, f_score_undersampled, precision_undersampled, recall_undersampled]
model_eval_dict_undersampled = {evaluate_df.columns[i]: model_eval_data_undersampled[i] for i in range(len(model_eval_data_undersampled))}
evaluate_df = evaluate_df.append(model_eval_dict_undersampled, ignore_index=True)


In [ ]:
# Print the evaluation dataframe
evaluate_df

In [ ]:
x = pd.DataFrame(X)
y = pd.DataFrame(y)

In [ ]:
# # Scaling the values
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [ ]:
#### Logistic Regression #####

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X,y)

In [ ]:
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

y_train.value_counts()

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=42)
model = lr.fit(X_train, y_train.values.ravel())  # Use y_train.values.ravel()
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
#evaluating the model
model_name = 'Logistic Regression - without balancing'
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)

acc_score = accuracy_score(y_test,y_test_pred)
f_score = f1_score(y_test, y_test_pred, average='weighted')
precision = precision_score(y_test, y_test_pred)
recall = metrics.recall_score(y_test,y_test_pred)
#creating a dataframe to compare the performance of different models
model_eval_data = [[model_name, train_score, test_score, acc_score, f_score, precision, recall]]
evaluate_df = pd.DataFrame(model_eval_data, columns=['Model Name', 'Training Score', 'Testing Score', 'Accuracy',
                                          'F1 Score', 'Precision', 'Recall'])
evaluate_df

In [ ]:
# renaming X_test and y_test variables
test_input = X_test.copy()
test_output = y_test.copy()
test_output.value_counts(normalize = True)

In [ ]:
#storing all fraud transactions
fraud_trans = df_num[df_num['isfraud'] == 1]
non_fraud_trans = df_num[df_num['isfraud'] == 0]

print('fraud data shape: ', fraud_trans.shape)
print('non fraud data shape: ', non_fraud_trans.shape)

#printing fraud data percentage
print('Fraud Data percentage: ', 100*(len(fraud_trans)/len(non_fraud_trans)))

In [ ]:
#modules for model building
#algorithms for sampling
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [ ]:
# Random UnderSampling
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X_train,y_train)
y_rus.value_counts()

In [ ]:
# Train - test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size = 0.3, random_state = 42, stratify = y_rus)
y_train.value_counts()

In [ ]:
# Creating a baseline linear model
lr = LogisticRegression(random_state=42)
# Model
model = lr.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_train_pred

In [ ]:
#predicting on imbalanced test data
test_pred=model.predict(test_input)
test_pred

In [ ]:
#printing the classification report of the model
print(classification_report(test_output,test_pred))

In [ ]:
#evaluating the model
model_name = 'Logistic Regression - Random Under Sampling'
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)

acc_score = accuracy_score(test_output,test_pred)
f_score = f1_score(test_output, test_pred, average='weighted')
precision = precision_score(test_output, test_pred)
recall = metrics.recall_score(test_output,test_pred)
#adding claculations to dataframe
model_eval_data = [model_name, train_score, test_score, acc_score, f_score, precision, recall]
model_eval_dict = {evaluate_df.columns[i]:model_eval_data[i] for i in range(len(model_eval_data))}
evaluate_df = evaluate_df.append(model_eval_dict, ignore_index=True)

evaluate_df

In [ ]:
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X_train, y_train)
y_ros.value_counts()

In [ ]:
X_train, X_test,y_train,y_test = train_test_split(X_ros, y_ros, test_size = 0.3, stratify = y_ros, random_state = 42)
y_train.value_counts()

In [ ]:
## Oversampling - SMOTE method
smote = SMOTE()
X_sm,y_sm = smote.fit_resample(X_train.astype('float'),y_train)
y_sm.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sm,y_sm,test_size = 0.3, random_state = 42, stratify = y_sm)
y_train.value_counts()

In [ ]:
#implementing logistic regression
lr = LogisticRegression(random_state=42)
#creating model
model = lr.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_train_pred

In [ ]:
test_pred = model.predict(test_input)
test_pred

In [ ]:
#printing classification report
print(classification_report(test_output, test_pred))

In [ ]:
#evaluating the model
model_name = 'Logistic Regression - SMOTE'
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)

acc_score = accuracy_score(test_output,test_pred)
f_score = f1_score(test_output, test_pred, average='weighted')
precision = precision_score(test_output, test_pred)
recall = metrics.recall_score(test_output,test_pred)
#adding claculations to dataframe
model_eval_data = [model_name, train_score, test_score, acc_score, f_score, precision, recall]
model_eval_dict = {evaluate_df.columns[i]:model_eval_data[i] for i in range(len(model_eval_data))}
evaluate_df = evaluate_df.append(model_eval_dict, ignore_index=True)

evaluate_df

In [ ]:
# Decision tree without sampling
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

dtree = DecisionTreeClassifier(max_depth=10)

model = dtree.fit(X_train,y_train)

y_test_pred = model.predict(X_test)

print(classification_report(y_test, y_test_pred))

#evaluating the model
model_name = 'Decision Tree - without balancing'
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)

acc_score = accuracy_score(y_test,y_test_pred)
f_score = f1_score(y_test, y_test_pred, average='weighted')
precision = precision_score(y_test, y_test_pred)
recall = metrics.recall_score(y_test,y_test_pred)
#adding claculations to dataframe
model_eval_data = [model_name, train_score, test_score, acc_score, f_score, precision, recall]
model_eval_dict = {evaluate_df.columns[i]:model_eval_data[i] for i in range(len(model_eval_data))}
evaluate_df = evaluate_df.append(model_eval_dict, ignore_index=True)

evaluate_df

In [ ]:
# Decision Tree Undersampling
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X_rus,y_rus, test_size=0.3, random_state=42)

dtree = DecisionTreeClassifier(max_depth=10)

model = dtree.fit(X_train,y_train)

test_pred = model.predict(test_input)

print(classification_report(test_output, test_pred))

#evaluating the model
model_name = 'Decision Tree - Random Under Sampling'
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)

acc_score = accuracy_score(test_output,test_pred)
f_score = f1_score(test_output, test_pred, average='weighted')
precision = precision_score(test_output, test_pred)
recall = metrics.recall_score(test_output,test_pred)
#adding claculations to dataframe
model_eval_data = [model_name, train_score, test_score, acc_score, f_score, precision, recall]
model_eval_dict = {evaluate_df.columns[i]:model_eval_data[i] for i in range(len(model_eval_data))}
evaluate_df = evaluate_df.append(model_eval_dict, ignore_index=True)

evaluate_df

In [ ]:
#Decision tree SMOTE
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X_sm,y_sm, test_size=0.3, random_state=42)

dtree = DecisionTreeClassifier(max_depth=10)

model = dtree.fit(X_train,y_train)

test_pred = model.predict(test_input)

print(classification_report(test_output, test_pred))

#evaluating the model
model_name = 'Decision Tree - SMOTE'
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)

acc_score = accuracy_score(test_output,test_pred)
f_score = f1_score(test_output, test_pred, average='weighted')
precision = precision_score(test_output, test_pred)
recall = metrics.recall_score(test_output,test_pred)
#adding claculations to dataframe
model_eval_data = [model_name, train_score, test_score, acc_score, f_score, precision, recall]
model_eval_dict = {evaluate_df.columns[i]:model_eval_data[i] for i in range(len(model_eval_data))}
evaluate_df = evaluate_df.append(model_eval_dict, ignore_index=True)

evaluate_df

In [ ]:
#Random Forest without sampling
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=100, criterion='gini')

model = rf.fit(X_train,y_train)

y_test_pred = model.predict(X_test)

print(classification_report(y_test, y_test_pred))

#evaluating the model
model_name = 'Random Forest - without balancing'
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)

acc_score = accuracy_score(y_test,y_test_pred)
f_score = f1_score(y_test, y_test_pred, average='weighted')
precision = precision_score(y_test, y_test_pred)
recall = metrics.recall_score(y_test,y_test_pred)
#adding claculations to dataframe
model_eval_data = [model_name, train_score, test_score, acc_score, f_score, precision, recall]
model_eval_dict = {evaluate_df.columns[i]:model_eval_data[i] for i in range(len(model_eval_data))}
evaluate_df = evaluate_df.append(model_eval_dict, ignore_index=True)

evaluate_df

In [ ]:
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X_rus,y_rus, test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=100, criterion='gini')

model = rf.fit(X_train,y_train)

test_pred = model.predict(test_input)

print(classification_report(test_output, test_pred))

#evaluating the model
model_name = 'Random Forest - Random Under Sampling'
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)

acc_score = accuracy_score(test_output,test_pred)
f_score = f1_score(test_output, test_pred, average='weighted')
precision = precision_score(test_output, test_pred)
recall = metrics.recall_score(test_output,test_pred)
#adding claculations to dataframe
model_eval_data = [model_name, train_score, test_score, acc_score, f_score, precision, recall]
model_eval_dict = {evaluate_df.columns[i]:model_eval_data[i] for i in range(len(model_eval_data))}
evaluate_df = evaluate_df.append(model_eval_dict, ignore_index=True)

evaluate_df

In [ ]:
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X_sm,y_sm, test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=100, criterion='gini')

model = rf.fit(X_train,y_train)

test_pred = model.predict(test_input)

print(classification_report(test_output, test_pred))

#evaluating the model
model_name = 'Random Forest - SMOTE'
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)

acc_score = accuracy_score(test_output,test_pred)
f_score = f1_score(test_output, test_pred, average='weighted')
precision = precision_score(test_output, test_pred)
recall = metrics.recall_score(test_output,test_pred)
#adding claculations to dataframe
model_eval_data = [model_name, train_score, test_score, acc_score, f_score, precision, recall]
model_eval_dict = {evaluate_df.columns[i]:model_eval_data[i] for i in range(len(model_eval_data))}
evaluate_df = evaluate_df.append(model_eval_dict, ignore_index=True)

evaluate_df

In [ ]:
sm = SMOTE()
X_train_new, y_train_new = sm.fit_resample(X_train, y_train.ravel())

# to demonstrate the effect of SMOTE over imbalanced datasets
fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize =(15, 5))
ax1.set_title('Before SMOTE')
pd.Series(y_train).value_counts().plot.bar(ax=ax1)

ax2.set_title('After SMOTE')
pd.Series(y_train_new).value_counts().plot.bar(ax=ax2)

plt.show()